# Validación cruzada

## Asignación estratificada aleatoria (ejemplo con dos clases), construccion manual

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.metrics import classification_report

In [2]:
#Leer los datos, nombre de los atributos y la clase
setInicial=pd.read_csv('diabetes.csv')

In [3]:
atributosName=setInicial.columns[:-1]
atributoClase=setInicial.columns[-1]
clasesName=setInicial[setInicial.columns[-1]].drop_duplicates()
print(atributosName)
print(clasesName)

Index(['preg', 'plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age'], dtype='object')
0      tested_positive
267    tested_negative
Name: class, dtype: object


In [4]:
#Determinar el número de pliegues
K_folds=3

#Genera una columna con un número aleatorio
setInicial['Tmp']=pd.DataFrame(np.random.rand(len(setInicial), 1))

#Separar el conjunto de acuerdo a las clases, además se ordenan por
#medio del atributo aleatorio para separarlos más adelante
Positivos=setInicial[setInicial['class']=='tested_positive'].sort_values(by='Tmp')
Positivos.pop('Tmp')
Negativos=setInicial[setInicial['class']=='tested_negative'].sort_values(by='Tmp')
Negativos.pop('Tmp')

#Calcular el número de instancias por pliegue
NumPositivos=int(len(Positivos)/K_folds)
NumNegativos=int(len(Negativos)/K_folds)

In [5]:
from sklearn.neural_network import MLPClassifier

clasificador = MLPClassifier(solver='lbfgs', 
                    alpha=1e-5,
                    hidden_layer_sizes=(8), 
                    random_state=42)

#En cada iteración se genera un train y test
contPos=0
contNeg=0
f1=[]

for x in range(1,K_folds+1,1):
    print("Pliegue: ",x)

    if x==K_folds:
        TopePos=len(Positivos)
        TopeNeg=len(Negativos)
    else:
        TopePos=contPos+NumPositivos
        TopeNeg=contNeg+NumNegativos

    #Generar el train de acuerdo a los índices de cada pliegue
    Test = pd.concat([Positivos.iloc[contPos:TopePos],Negativos.iloc[contNeg:TopeNeg]],axis=0)
    #Generar el test, tomando como referencia los índices que no aparecen en el train
    Training = pd.concat([Positivos.iloc[0:contPos],Positivos.iloc[TopePos:len(Positivos)],
                          Negativos.iloc[0:contNeg],Negativos.iloc[TopeNeg:len(Negativos)]],axis=0)
    
    print("Train: ",len(Training)," intancias.\nTest: ",len(Test)," instancias.")
    contPos+=NumPositivos
    contNeg+=NumNegativos

    #A PARTIR DE AQUÍ DE INICIA CON LA SEPARACIÓN Y CLASIFICACIÓN
    valoresTrain = Training[atributosName]
    #print(valoresTrain.head())
    valoresTest = Test[atributosName]
    clasesTrain = Training[atributoClase]
    #print(clasesTrain.head())
    clasesTest = Test[atributoClase]

    ################## Modelo ###########################
    modelo = clasificador.fit(valoresTrain, clasesTrain)

    ################## Clasificar  #################
    predict = modelo.predict(valoresTest)

    ############ Evaluar ###############################
    reporte=classification_report(clasesTest, predict, labels=clasesName, output_dict=True)
    reporte2=classification_report(clasesTest, predict, labels=clasesName)#, output_dict=True)
    print(reporte2)
    f1.append(reporte['accuracy'])
    
print(max(f1),sum(f1)/len(f1),min(f1))

Pliegue:  1
Train:  712  intancias.
Test:  354  instancias.
                 precision    recall  f1-score   support

tested_positive       0.71      0.03      0.05       177
tested_negative       0.50      0.99      0.67       177

       accuracy                           0.51       354
      macro avg       0.61      0.51      0.36       354
   weighted avg       0.61      0.51      0.36       354

Pliegue:  2
Train:  712  intancias.
Test:  354  instancias.
                 precision    recall  f1-score   support

tested_positive       0.76      0.69      0.73       177
tested_negative       0.72      0.79      0.75       177

       accuracy                           0.74       354
      macro avg       0.74      0.74      0.74       354
   weighted avg       0.74      0.74      0.74       354

Pliegue:  3
Train:  708  intancias.
Test:  358  instancias.
                 precision    recall  f1-score   support

tested_positive       0.75      0.68      0.71       179
tested_negative

/home/uziel/Programas/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
